<a href="https://colab.research.google.com/github/Lilchoto3/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/LS_DS_424_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [2]:
!pip install keras-tuner

In [4]:
##### Your Code Here #####

# Setup cell, will initialize and fit encoders
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

# <<< SETUP VARIABLES >>>

# get data
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

# standardize column names cause i'm lazy
df.columns = [col.lower() for col in df.columns]
# entirely unique and therefore worthless
df = df.drop(columns='customerid')
# fix totalcharges
df['totalcharges'] = df['totalcharges'].map({' ':0.0}, na_action='ignore')

# set up target and features
target = 'churn'
non_cat_feats = ['seniorcitizen','tenure','monthlycharges','totalcharges']
cat_feats = list(df.drop(columns=[target]+non_cat_feats).columns)
features = cat_feats + non_cat_feats

# <<< END SETUP VARIABLES >>>

# The following code shouldn't need to be modified
# init encoder for target and fit it
label = LabelEncoder()
label.fit(df[target])

# init encoders for each categorical column and fit each one seperately
ordinal = OrdinalEncoder()
ordinal.fit(df[cat_feats])

# ----------------------------------------------------------------------------

# preproccessing function, use this to encode data for modelling/predicting
# scores from models
def preprocess(df):
  '''
  Preprocessing function

  inputs: DataFrame

  outputs: X_train, y_train as numpy arrays
  '''
  # copy from original dataframe
  X = df[features].copy()
  y = df[target].copy()

  # transform target
  y = label.transform(y)

  # transform categoricals
  X_cat = ordinal.transform(X[cat_feats])
  # put transformed categoricals back into dataframe to maintain structure
  X_cat = pd.DataFrame(X_cat, columns=cat_feats)
  for col in cat_feats:
    X[col] = X_cat[col]
  for col in X.columns:
    X[col] = pd.to_numeric(X[col])
  # print(X.dtypes)
  # print(X['totalcharges'][:5])
  return np.asarray(X.values).astype(np.float32), np.asarray(y).astype(np.float32)

# ----------------------------------------------------------------------------

# preprocess the model
X_train, y_train = preprocess(df)
# print(X_train[:5])
# print(y_train[:5])

# set up the model
def create_model(layers=0, num_nodes=32, act_type='relu'):
  model = keras.models.Sequential()
  model.add(keras.layers.Dense(num_nodes, activation=act_type))
  for i in range(layers):
    model.add(keras.layers.Dense(num_nodes, activation=act_type))
  model.add(keras.layers.Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
  return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'layers': [0, 1, 2, 3],
              'num_nodes': [8, 16, 32, 64, 128],
              'act_type': ['relu','sigmoid','selu','tanh']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, verbose=2)
grid_result = grid.fit(X_train, y_train)
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] act_type=relu, layers=0, num_nodes=8 ............................
[CV] ............. act_type=relu, layers=0, num_nodes=8, total=   0.9s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV] act_type=relu, layers=0, num_nodes=8 ............................
[CV] ............. act_type=relu, layers=0, num_nodes=8, total=   0.9s
[CV] act_type=relu, layers=0, num_nodes=8 ............................
[CV] ............. act_type=relu, layers=0, num_nodes=8, total=   0.9s
[CV] act_type=relu, layers=0, num_nodes=8 ............................
[CV] ............. act_type=relu, layers=0, num_nodes=8, total=   0.9s
[CV] act_type=relu, layers=0, num_nodes=8 ............................
[CV] ............. act_type=relu, layers=0, num_nodes=8, total=   0.9s
[CV] act_type=relu, layers=0, num_nodes=16 ...........................
[CV] ............ act_type=relu, layers=0, num_nodes=16, total=   0.9s
[CV] act_type=relu, layers=0, num_nodes=16 ...........................
[CV] ............ act_type=relu, layers=0, num_nodes=16, total=   1.3s
[CV] act_type=relu, layers=0, num_nodes=16 ...........................
[CV] ............ act_type=relu, layers=0, num_nodes=16, total=   0.9s
[CV] a

[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  8.1min finished


Best: 0.7346286535263061 using {'act_type': 'relu', 'layers': 0, 'num_nodes': 8}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 0, 'num_nodes': 8}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 0, 'num_nodes': 16}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 0, 'num_nodes': 32}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 0, 'num_nodes': 64}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 0, 'num_nodes': 128}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 1, 'num_nodes': 8}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 1, 'num_nodes': 16}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 1, 'num_nodes': 32}
Means: 0.734628653526306

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?